In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from typing import List
from tqdm import tqdm

In [ ]:
# !pip install implicit

!!!Нужно выполнить все ячейки, финальная метрика в самой последней

#распаковать данные и провести анализ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_users = pd.read_csv('/content/drive/MyDrive/2025Mag2/RecSys/DataSet/users.csv')
df_items = pd.read_csv('/content/drive/MyDrive/2025Mag2/RecSys/DataSet/items.csv')
df_interactions = pd.read_csv('/content/drive/MyDrive/2025Mag2/RecSys/DataSet/interactions.csv')

In [ ]:
# df_users = pd.read_csv('/content/users.csv')
# df_items = pd.read_csv('/content/items.csv')
# df_interactions = pd.read_csv("/content/interactions.csv")

# df_interactions = df_interactions.head(int(len(df_interactions)/2))

In [ ]:
df_items.head(5)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [ ]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15963 entries, 0 to 15962
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       15963 non-null  int64  
 1   content_type  15963 non-null  object 
 2   title         15963 non-null  object 
 3   title_orig    11218 non-null  object 
 4   release_year  15865 non-null  float64
 5   genres        15963 non-null  object 
 6   countries     15926 non-null  object 
 7   for_kids      566 non-null    float64
 8   age_rating    15961 non-null  float64
 9   studios       1065 non-null   object 
 10  directors     14454 non-null  object 
 11  actors        13344 non-null  object 
 12  description   15961 non-null  object 
 13  keywords      15540 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 1.7+ MB


In [ ]:
df_users.head(5)

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [ ]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840197 entries, 0 to 840196
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   840197 non-null  int64 
 1   age       826102 non-null  object
 2   income    825421 non-null  object
 3   sex       826366 non-null  object
 4   kids_flg  840197 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 32.1+ MB


In [ ]:
df_interactions['last_watch_dt'] = pd.to_datetime(df_interactions['last_watch_dt'])
print(len(df_interactions))
# удалить ломанную строку
df_interactions.drop(df_interactions.tail(1).index,inplace=True)
print(len(df_interactions))
df_interactions = df_interactions.assign(reallyWatched = df_interactions['watched_pct'] >= 10.0)

5476250
5476249


In [ ]:
df_interactions.head(5)

,user_id,item_id,last_watch_dt,total_dur,watched_pct,reallyWatched
0,176549,9506,2021-05-11,4250,72.0,True
1,699317,1659,2021-05-29,8317,100.0,True
2,656683,7107,2021-05-09,10,0.0,False
3,864613,7638,2021-07-05,14483,100.0,True
4,964868,9506,2021-04-30,6725,100.0,True


In [ ]:
df_interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int64         
 1   item_id        int64         
 2   last_watch_dt  datetime64[ns]
 3   total_dur      int64         
 4   watched_pct    float64       
 5   reallyWatched  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3)
memory usage: 214.1 MB


#Сформировать набор данных

In [ ]:
# параметр K
k = 10

test_size_days = 7
dateColumnName = "last_watch_dt"
userColumnName = "user_id"
watchdPrcntColumnName = "watched_pct"
itemIdColumnName = "item_id"
totalDurColumnName = "total_dur"
reallyWatchedColumn = "reallyWatched"

In [ ]:
from datetime import datetime, timedelta

# Получение даты с которой начинается тестовый набор
def getLastWeekFirstDay(lastDate, test_size_days=7):
    return lastDate - timedelta(days=(test_size_days-1))

In [ ]:
# получение тренировочного и тестового наборов по дням
def splitByLastDate(df_interactions, test_size_days = 7):
  # Получаем последнюю дату
  maxDate = df_interactions[dateColumnName].max()
  lastWeekFirstDate = getLastWeekFirstDay(maxDate, test_size_days)
  print(lastWeekFirstDate)
  data_train = df_interactions[df_interactions[dateColumnName] < lastWeekFirstDate]
  data_test = df_interactions[df_interactions[dateColumnName] >= lastWeekFirstDate]
  return data_train, data_test

In [ ]:
df_interactions_train, df_interactions_test = splitByLastDate(df_interactions, test_size_days)

2021-08-16 00:00:00


In [ ]:
df_interactions_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 424436 entries, 9 to 5476248
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   user_id        424436 non-null  int64         
 1   item_id        424436 non-null  int64         
 2   last_watch_dt  424436 non-null  datetime64[ns]
 3   total_dur      424436 non-null  int64         
 4   watched_pct    424435 non-null  float64       
 5   reallyWatched  424436 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3)
memory usage: 19.8 MB


In [ ]:
df_interactions_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5051815 entries, 0 to 5476250
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int64         
 1   item_id        int64         
 2   last_watch_dt  datetime64[ns]
 3   total_dur      int64         
 4   watched_pct    float64       
 5   reallyWatched  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3)
memory usage: 236.1 MB


In [ ]:
def filter_userInteractions(df):
  # df_Temp = df.copy()
  # df_Temp["grouped"] = df.groupby(userColumnName)[itemIdColumnName].unique().reset_index()
  df_Temp = df.groupby(userColumnName)[itemIdColumnName].unique().reset_index()
  df_Temp.columns = [userColumnName, itemIdColumnName]
  filtered_df_Temp = df_Temp[df_Temp[itemIdColumnName].apply(lambda x: len(x)>2)]
  unique_users = filtered_df_Temp[userColumnName].unique()
  print(len(unique_users))
  return df[df[itemIdColumnName].isin(unique_users)]



In [ ]:
df_interactions_train = filter_userInteractions(df_interactions_train)


429380


In [ ]:
df_interactions_test = filter_userInteractions(df_interactions_test)

48974


In [ ]:
df_interactions_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20279 entries, 141 to 5476029
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        20279 non-null  int64         
 1   item_id        20279 non-null  int64         
 2   last_watch_dt  20279 non-null  datetime64[ns]
 3   total_dur      20279 non-null  int64         
 4   watched_pct    20279 non-null  float64       
 5   reallyWatched  20279 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3)
memory usage: 970.4 KB


In [ ]:
df_interactions_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2243750 entries, 2 to 5476250
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int64         
 1   item_id        int64         
 2   last_watch_dt  datetime64[ns]
 3   total_dur      int64         
 4   watched_pct    float64       
 5   reallyWatched  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3)
memory usage: 104.9 MB


In [ ]:
# достать уникальных пользователей из train
train_unique_users = df_interactions_train[userColumnName].unique()
test_unique_users = df_interactions_test[userColumnName].unique()

test_unique_items = df_interactions_test[itemIdColumnName].unique()
train_unique_items = df_interactions_train[itemIdColumnName].unique()

In [ ]:
len(test_unique_users)

18010

In [ ]:
# отфильтровать валидные случаи
user_tests_content_interaction = df_interactions_test[((df_interactions_test[reallyWatchedColumn] == True))]

In [ ]:
user_tests_content_interaction_Really = df_interactions_test[((df_interactions_test[reallyWatchedColumn] == True))]
user_train_content_interaction_Really = df_interactions_train[((df_interactions_train[reallyWatchedColumn] == True))]

In [ ]:
test_unique_items_really = user_tests_content_interaction_Really[itemIdColumnName].unique()
train_unique_items_really = user_train_content_interaction_Really[itemIdColumnName].unique()

In [ ]:
user_tests_content_interaction = df_interactions_test.groupby(userColumnName)[itemIdColumnName].unique().reset_index()

In [ ]:
user_tests_content_interaction.columns = [userColumnName, itemIdColumnName]

In [ ]:
user_tests_content_interaction

,user_id,item_id
0,1,[3669]
1,106,"[12908, 337]"
2,157,[16228]
3,181,"[4880, 16228]"
4,235,[4495]
...,...,...
18005,1097328,[16228]
18006,1097444,[5669]
18007,1097471,[1785]
18008,1097473,[12908]


In [ ]:
filtered_user_tests_content_interaction = user_tests_content_interaction[user_tests_content_interaction[itemIdColumnName].apply(lambda x: len(x)>1)]


In [ ]:
filtered_user_tests_content_interaction

,user_id,item_id
1,106,"[12908, 337]"
3,181,"[4880, 16228]"
25,1180,"[13243, 3182]"
35,2041,"[10119, 13979]"
37,2094,"[7789, 12908]"
...,...,...
17966,1094750,"[13167, 14222]"
17977,1095299,"[16228, 11237]"
17978,1095411,"[2954, 15038]"
17998,1096807,"[516, 16397, 9404]"


In [ ]:
len(filtered_user_tests_content_interaction)

5

In [ ]:
def precision(recommended_list, bought_list):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision


def precision_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    #assert len(bought_list) > len(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)


    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):

    # your_code
    # Лучше считать через скалярное произведение, а не цикл
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    precision = np.dot(flags, bought_list) / np.sum(recommended_list)
    return precision

In [ ]:
def recall(recommended_list, bought_list):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list)

    recall = flags.sum() / len(bought_list)

    return recall


def recall_at_k(recommended_list, bought_list, k=5):

    # your_code
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])

    flags = np.isin(bought_list, recommended_list)

    recall = flags.sum() / len(bought_list)

    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):

    # your_code
    bought_list = np.array(bought_list)
    recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    recall = np.dot(flags, bought_list) / np.sum(bought_list)

    return recall

In [ ]:
def ap_k(recommended_list, bought_list, k=5):
    # print("=",bought_list, recommended_list)
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(recommended_list, bought_list)
    # if (np.isin([True],flags)[0]):
    #   print(flags)
    if sum(flags) == 0:
        return 0

    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k

    result = sum_ / sum(flags)

    return result

In [ ]:
# получение списка взаимодействий юзера
def getUserItems(userInteractionItems, user_id):
  return userInteractionItems[(userInteractionItems[userColumnName] == user_id)][itemIdColumnName].values[0]

In [ ]:
# получение значения из матрицы по двум столбцам
def getColFromBy2Col(Items, first_column, second_column, first_c_id):
  return Items[(Items[first_column] == first_c_id)][second_column]

In [ ]:
# получение значения из матрицы по двум столбцам
def getValueFromBy2Col(Items, first_column, second_column, first_c_id):
  colFromBy2Col = getColFromBy2Col(Items, first_column, second_column, first_c_id)
  # print(colFromBy2Col)
  if len(colFromBy2Col) > 0:
    return colFromBy2Col.values[0]
  else:
    return None

#random

In [ ]:
# def getRandomRecommendOLD(items, k=10):
#   return items[itemIdColumnName].sample(k,replace=True).tolist()

# def getRandomRecommendOLDOLD(items, k=10):
#   return pd.Series(items[itemIdColumnName].unique()).sample(k,replace=True).tolist()

def getRandomRecommend(items, k=10):
  return np.random.choice(items, k)

In [ ]:
def getFirstK(items, k=10):
  return items[:k]

In [ ]:
# item = рекомендации взять из тестового набора
def map_k(items, userContentInteractions, func, k=10):
    user_ids = userContentInteractions[userColumnName].tolist()
    result = [ap_k(func(items, k), getUserItems(userContentInteractions, i)) for i in user_ids]
    return np.average(result)

#popular

In [ ]:
# Получение популярных items по всем
def getPopularItems(df):
  items_content_interaction_Pop = df.groupby(itemIdColumnName)[watchdPrcntColumnName].unique().reset_index()
  items_content_interaction_Pop.columns = [itemIdColumnName, watchdPrcntColumnName]
  items_content_interaction_Pop[watchdPrcntColumnName] = items_content_interaction_Pop[watchdPrcntColumnName].apply(lambda x: len(np.array(x)))
  items_content_interaction_Pop.sort_values(by=watchdPrcntColumnName, ascending=False)
  return items_content_interaction_Pop[itemIdColumnName].unique()

#Popular+UPopular

In [ ]:
def sumColVal(x):
  print(x)
  return (len(x[1]))

#ЗАДАЧА 2

In [ ]:
# all_users = df_interactions[userColumnName].unique()


In [ ]:
# получить интеракции пользователя
def get_user_interactions(user_id, df_interactions):
  return df_interactions[((df_interactions[userColumnName] == user_id))]

def getUniqueColumnValues(df, columnName):
  return df[columnName].unique()

In [ ]:
import random

#Получение случайного пользователя со списком взаимодействий > k
def getRandomUser(df_interactions, k = 10):
  random_user_interactions = []
  while len(random_user_interactions) < k:
    random_user = random.choice(all_users)
    random_user_interactions = get_user_interactions(random_user, df_interactions)
  return random_user_interactions

In [ ]:
# получение строк только по вхождениям из списка
def getitemsByList(df,columnName, list_of_users):
  return df[df[columnName].isin(list_of_users)]

In [ ]:
releaseYearCol = 'release_year'
genresCol = 'genres'
actorsCol = 'actors'
directorsCol = 'directors'

In [ ]:
def getArrayFromDF(df, columnName):
  return df[columnName].apply(lambda x: x.split(', '))

In [ ]:
# Условия для itemXitem

# тот-же год
def sameYearDF(df, year):
  conditions = [
    (df[releaseYearCol] == year),
    (df[releaseYearCol] != year)
  ]
  choices = [0.25, 0]
  return np.select(conditions, choices)

# Хотя бы один жанр
def sameGenreDF(df, genres):
  conditions = [
    (df[genresCol].str.contains('|'.join(genres)) == True),
    (df[genresCol].str.contains('|'.join(genres)) != True)
  ]
  choices = [0.25, 0]
  return np.select(conditions, choices)

# Хотя бы один актер
def sameActorDF(df, actors):
  conditions = [
    (df[actorsCol].str.contains('|'.join(actors)) == True),
    (df[actorsCol].str.contains('|'.join(actors)) != True)
  ]
  choices = [0.25, 0]
  return np.select(conditions, choices)

# Хотя бы один режиссер
def sameDirectorDF(df, directors):
  conditions = [
    (df[directorsCol].str.contains('|'.join(directors)) == True),
    (df[directorsCol].str.contains('|'.join(directors)) != True)
  ]
  choices = [0.25, 0]
  return np.select(conditions, choices)

##############################################
# Условия для userXitem

# # Сравнить по проценту просмотренного
# def compareWatchedDF(df, watchPrcnt):
#   conditions = [
#     (df[watchdPrcntColumnName] >= watchPrcnt),
#     (df[watchdPrcntColumnName] < watchPrcnt)
#   ]
#   choices = [1, 0]
#   return np.select(conditions, choices)

# Сравнить по проценту просмотренного и по диапазону дат (и по полу пользователя)
def sameDateWatchedDF(df, df_u, time_from, time_to, watchPrcnt, sex):
  conditions = [
    ((df[dateColumnName] > time_from) & (df[dateColumnName] < time_to) & (df[watchdPrcntColumnName] >= 15.0) & (df_u['sex'] == sex) == True),
    ((df[dateColumnName] > time_from) & (df[dateColumnName] < time_to) & (df[watchdPrcntColumnName] >= 15.0) & (df_u['sex'] == sex) != True)
  ]
  choices = [1, 0]
  return np.select(conditions, choices)

# # Сравнить по проценту просмотренного и по диапазону дат
# def similarUser(df, sex):



In [ ]:
def getCoeff(b,itemId, c = 0.25):
  # b.replace(np.nan, False, inplace=True)
  # print(c if b.get(itemId) else 0)
  # print(b.size)
  return c if b.get(itemId) == True else 0

In [ ]:
def convertToArray(df, col1, col2, id):
  data = getValueFromBy2Col(df, col1, col2, id)
  return data.split(', ') if type(data) == 'str'else []


In [ ]:
def map_k_KNN_users_test(df_interactions_test, df_interactions_train, df_items, df_users, k=10, interrupt = False):
  # получаем всех юзеров из тестового и тренировочного набора
  all_test_users = getUniqueColumnValues(df_interactions_test, userColumnName)
  all_train_users = getUniqueColumnValues(df_interactions_train, userColumnName)
  # all_test_users = [all_test_users[1814]]
  # Получаем списко самых популярных впринципе из Train
  list_all_popular = getPopularItems(df_interactions_train)

  # Все
  # user_tests_content_interaction = df_interactions_test.groupby(userColumnName)[itemIdColumnName].unique().reset_index()
  # user_tests_content_interaction.columns = [userColumnName, itemIdColumnName]

  sum_ap_k = 0 # сумма значений метрики ap k
  userRecomenadations = 0 # просмотренные пользователи с рекомендациями
  counter = 0
  len_test = len(all_test_users)
  # для каждого пользователя находим рекоммендации
  for user_id in all_test_users:

    counter += 1
    all_test_items = getColFromBy2Col(df_interactions_test, userColumnName, itemIdColumnName, user_id).unique()

    # если в тесте меньше 5 взаимодействий, значит и рекомендовать ему нечего
    if len(all_test_items) > 5:
      # получаем все items из train у юзера
      all_train_items = getColFromBy2Col(df_interactions_train, userColumnName, itemIdColumnName, user_id).unique()
      # Если нету такого пользователя в Train или мало , то рекомендуем ему популярные ('Холодный старт')
      is_all_train_items = len(all_train_items) > 0
      recomendations = []
      if (is_all_train_items):
        user_interactions_train = get_user_interactions(user_id, df_interactions_train)
        user_Item_Train_df = pd.DataFrame(df_interactions_train[userColumnName])

        # Заполняем матрицу user+item
        # 3 недели
        time_delta_days = np.timedelta64(21, 'D')

        for itemId in all_train_items:
          time_from = getValueFromBy2Col(user_interactions_train, itemIdColumnName, dateColumnName, itemId) - time_delta_days
          time_to = getValueFromBy2Col(user_interactions_train, itemIdColumnName, dateColumnName, itemId) + time_delta_days
          user_sex = getValueFromBy2Col(df_users, userColumnName, 'sex', user_id)
          watchPrcnt = getValueFromBy2Col(user_interactions_train, itemIdColumnName, watchdPrcntColumnName, itemId)
          # print(time_from, time_to, watchPrcnt)
          isSameDateWtchdDF = sameDateWatchedDF(df_interactions_train, df_users, time_from, time_to, watchPrcnt, user_sex)
          # user_Item_Train_df.insert(1, itemId, pd.Series(isSameDateWtchdDF))
          values_uI = pd.Series(isSameDateWtchdDF)
          user_Item_Train_df = pd.concat([user_Item_Train_df, values_uI.rename(itemId)], axis=1)


          # user_Item_Train_df[itemId] = pd.Series(isSameDateWtchdDF)
          # user_Item_Train_df[itemId] = np.where((df_interactions_train[itemIdColumnName] == itemId) & (df_interactions_train[watchdPrcntColumnName] > watchPrcnt) & (df_interactions_train[dateColumnName] > time_from) & (df_interactions_train[dateColumnName] < time_to), 1, 0)
        cols_items = all_train_items
        # print(cols_items)
        # user_Item_Train_df.info()
        # print()
        # берем тех юзеров, у которых совпадение хотя бы по одному фильму
        temp_df_user_item = user_Item_Train_df[user_Item_Train_df[cols_items].sum(axis=1) >= 1]
        # подсчитаем сумму всех совпадений по экземплярам и отсортируем
        # print(cols_items)
        # print(temp_df_user_item.head(5))
        if len(cols_items) > 1:
          # temp_df_user_item["sumCount"] = temp_df_user_item[cols_items].sum(axis=1)
          # temp_df_user_item.assign(sumCount = temp_df_user_item[cols_items].sum(axis=1))
          temp_df_user_item.insert(1, 'sumCount', temp_df_user_item[cols_items].sum(axis=1))
        else:
          temp_df_user_item["sumCount"] = temp_df_user_item[cols_items[0]]
        temp_df_user_item.sort_values(by='sumCount', ascending=False)

        unique_users_List_Train = temp_df_user_item[userColumnName].unique()
        unique_user_Interactions_Train = getitemsByList(df_interactions_train, userColumnName, unique_users_List_Train)
        unique_user_Interactions_Train = unique_user_Interactions_Train.drop_duplicates(subset=[itemIdColumnName], keep=False)  # С использованием keep=False дубликаты не сохраняются!
        # len(unique_user_Interactions_Train)
        selected_items_train_df = getitemsByList(df_items, itemIdColumnName, unique_user_Interactions_Train[itemIdColumnName].unique())
        unique_user_Interactions_Train_df = pd.DataFrame(selected_items_train_df[itemIdColumnName])

        # получаем матрицу item+item
        for itemId in all_train_items:
          release_year = getValueFromBy2Col(df_items, itemIdColumnName, releaseYearCol, itemId)
          genres = convertToArray(df_items, itemIdColumnName, genresCol, itemId)
          actors = convertToArray(df_items, itemIdColumnName, actorsCol, itemId)
          directors = convertToArray(df_items, itemIdColumnName, directorsCol, itemId)

          isSameGenre = sameGenreDF(selected_items_train_df,genres)
          isSameYear = sameYearDF(selected_items_train_df,release_year)
          isSameActor = sameActorDF(selected_items_train_df,actors)
          isSameDirector = sameDirectorDF(selected_items_train_df,directors)
          values = pd.Series(isSameGenre + isSameYear + isSameActor + isSameDirector)

          # unique_user_Interactions_Train_df[itemId] = values
          unique_user_Interactions_Train_df = pd.concat([unique_user_Interactions_Train_df, values.rename(itemId)], axis=1)
          # unique_user_Interactions_Train_df.insert(1, itemId, values)


        # берем тех items, у которых совпадение в сумме дает 1
        temp_df_item_item = unique_user_Interactions_Train_df[unique_user_Interactions_Train_df[cols_items].sum(axis=1) >= 1]
        # сортируем
        # print(cols_items)
        # print(temp_df_item_item.head(5))

        if len(cols_items) > 1:
          # temp_df_item_item["sumCount"] = temp_df_item_item[cols_items].sum(axis=1)
          # temp_df_item_item.assign(sumCount = temp_df_item_item[cols_items].sum(axis=1))
          temp_df_item_item.insert(1, 'sumCount', temp_df_item_item[cols_items].sum(axis=1))
        else:
          temp_df_item_item["sumCount"] = temp_df_item_item[cols_items[0]]
        temp_df_item_item.sort_values(by='sumCount', ascending=False)

        # топ k
        recomendations = temp_df_item_item.head(k)[itemIdColumnName].unique()
        # если все рекомендации то считаем ap_k
        if len(recomendations) == k:
          sum_ap_k += ap_k(recomendations, all_test_items)

      # Если нету такого пользователя в Train, то рекомендуем ему популярные ('Холодный старт')
      # Или если в рекомедациях не набралось k экземпляров, дополняем их популярными
      if (is_all_train_items == False or len(recomendations)<k):
        dif_k = k - len(recomendations)
        list_recommend = []
        # дополняем список рекомендуемых экземплярами из популярного
        if dif_k>0:
          list_recommend = np.concatenate([recomendations, getFirstK(list_all_popular, dif_k)])
        else:
          list_recommend = recomendations
        sum_ap_k += ap_k(list_recommend, all_test_items)

      # + счетчик рекомендаций
      userRecomenadations += 1
    # if (counter % 50 == 0):
    print('_____________________',counter, ' / [',userRecomenadations,' ]/ ', len_test)
    if interrupt == True and userRecomenadations == 100:
      break

  return sum_ap_k/userRecomenadations



In [ ]:
# mapk_KNN = map_k_KNN_users_test(df_interactions_test, df_interactions_train, df_items, df_users, k)

In [ ]:
# mapk_KNN

In [ ]:
# 1814

1814

# черновик

In [ ]:
# random_user_interactions = getRandomUser(df_interactions, k)

In [ ]:
# random_user_interactions.head(5)

In [ ]:
# len(random_user_interactions)

In [ ]:
# random_user_interactions_train, random_user_interactions_test = splitByLastDate(random_user_interactions, test_size_days)

In [ ]:
# random_user_interactions_train.info()

In [ ]:
# random_user_interactions_test.info()

In [ ]:
# df = pd.DataFrame({'col_1': [100, 101, 103],
#                    'col_2': [95, 94, 97]})

# df['col_3'] = df.col_2.div(df.col_1).between(0.95, 1.05)
# print(df)

In [ ]:
# unique_random_user_Items_Train = random_user_interactions_train[itemIdColumnName].unique()
# unique_random_user_Items_Train

In [ ]:
# user_random_Item_Train_df = pd.DataFrame(df_interactions_train[userColumnName])

In [ ]:
# user_random_Item_Train_df['user_id']

In [ ]:
# type(user_random_Item_Train_df)

In [ ]:
# # 3 недели
# time_delta_days = np.timedelta64(21, 'D')
# # print(time_from, time_to)

# for itemId in unique_random_user_Items_Train:
#   time_from = getValueFromBy2Col(random_user_interactions_train, itemIdColumnName, dateColumnName, itemId) - time_delta_days
#   time_to = getValueFromBy2Col(random_user_interactions_train, itemIdColumnName, dateColumnName, itemId) + time_delta_days
#   watchPrcnt = getValueFromBy2Col(random_user_interactions_train, itemIdColumnName, watchdPrcntColumnName, itemId)
#   print(time_from, time_to, watchPrcnt)

#   user_random_Item_Train_df[itemId] = np.where((df_interactions_train[itemIdColumnName] == itemId) & (df_interactions_train[watchdPrcntColumnName] > watchPrcnt) & (df_interactions_train[dateColumnName] > time_from) & (df_interactions_train[dateColumnName] < time_to), 1, 0)
#   # user_random_Item_Train_df[itemId] = np.where((df_interactions_train[itemIdColumnName] == itemId), 1, 0)

In [ ]:
# user_random_Item_Train_df

In [ ]:
# def lambdaAllSum(df, columns):
#   lambdaF = None
#   for i in columns:
#     lambdaF += df[i]
#   return lambdaF


In [ ]:
# user_random_Item_Train_df["sumSimilarity"] =

In [ ]:
# # user_random_Item_Train_df[(user_random_Item_Train_df != 0).all(1)]
# cols = unique_random_user_Items_Train
# tem_df = user_random_Item_Train_df[user_random_Item_Train_df[cols].sum(axis=1) >= 1]

In [ ]:
# tem_df

In [ ]:
# unique_users_List_Train = tem_df[userColumnName].unique()


In [ ]:
# unique_random_user_Interactions_Train = getitemsByList(df_interactions_train, userColumnName, unique_users_List_Train)

In [ ]:
# len(unique_random_user_Interactions_Train)

In [ ]:
# unique_random_user_Interactions_Train = unique_random_user_Interactions_Train.drop_duplicates(subset=[itemIdColumnName], keep=False)  # С использованием keep=False дубликаты не сохраняются!
# len(unique_random_user_Interactions_Train)

In [ ]:
# selected_items_train_df = getitemsByList(df_items, itemIdColumnName, unique_random_user_Interactions_Train[itemIdColumnName].unique() )

In [ ]:
# len(selected_items_train_df)

In [ ]:
# unique_random_user_Interactions_Train_df = pd.DataFrame(selected_items_train_df[itemIdColumnName])

In [ ]:
# unique_random_user_Interactions_Train_df

In [ ]:
# selected_items_train_df.info()


In [ ]:
# # 3 недели
# # time_delta_days = np.timedelta64(21, 'D')
# # print(time_from, time_to)

# for itemId in unique_random_user_Items_Train:
#   release_year = getValueFromBy2Col(df_items, itemIdColumnName, releaseYearCol, itemId)
#   genres = convertToArray(df_items, itemIdColumnName, genresCol, itemId)
#   actors = convertToArray(df_items, itemIdColumnName, actorsCol, itemId)
#   directors = convertToArray(df_items, itemIdColumnName, directorsCol, itemId)

#   # print(genres, actors, directors)
#   # watchPrcnt = getValueFromBy2Col(random_user_interactions_train, itemIdColumnName, watchdPrcntColumnName, itemId)
#   # print(time_from, time_to, watchPrcnt)
#   isSameGenre = sameGenreDF(selected_items_train_df,genres)
#   isSameYear = sameYearDF(selected_items_train_df,release_year)
#   isSameActor = sameActorDF(selected_items_train_df,actors)
#   isSameDirector = sameDirectorDF(selected_items_train_df,directors)
#   # isSameDirector = sameDirectorDF(selected_items_train_df,directors).fillna(False)
#   values =  pd.Series(isSameGenre + isSameYear + isSameActor + isSameDirector)
#   # print(type(values))
#   # user_random_Item_Train_df[itemId] = np.where((selected_items_train_df[releaseYearCol] == release_year) | sum(np.isin(getArrayFromDF(selected_items_train_df, genresCol)[genresCol], genres)), 1, 0)
#   # user_random_Item_Train_df[itemId] = np.where(sum(np.isin(getArrayFromDF(selected_items_train_df, genresCol), genres)), 1, 0)
#   # unique_random_user_Interactions_Train_df[itemId] = np.where((isSameGenre | isSameYear | isSameActor | isSameDirector), getCoeff(isSameGenre, itemId) + getCoeff(isSameYear,itemId) + getCoeff(isSameActor,itemId) + getCoeff(isSameDirector,itemId), 0)
#   # unique_random_user_Interactions_Train_df[itemId] = np.where((isSameGenre | isSameYear | isSameActor | isSameDirector), 1, 0)
#   unique_random_user_Interactions_Train_df[itemId] = values
#   # unique_random_user_Interactions_Train_df[itemId] = np.where((selected_items_train_df[releaseYearCol] == release_year), 1, 0)
#   # user_random_Item_Train_df[itemId] = np.where((df_interactions_train[itemIdColumnName] == itemId), 1, 0)

In [ ]:
# unique_random_user_Interactions_Train_df

In [ ]:
# unique_random_user_Interactions_Train_df

In [ ]:
# Нужно:
# 1) получить просмотренные фильмы пользователей
# 2) создать матрицу, где строки это эти фильмы, а стобцы - фильмы рандомного пользователя
# 3) по тому-же способу создать матрицу соседей, но с фильмами (тот же год) || (*по жанрам совпадение > 50%) || (есть любой актер из списка) || (есть такой-же режиссер): соотношение (25/25/25/25)

In [ ]:
# cols = unique_random_user_Items_Train
# tem_df = unique_random_user_Interactions_Train_df[unique_random_user_Interactions_Train_df[cols].sum(axis=1) >= 1]

In [ ]:
# tem_df['sum'] = tem_df[cols].sum(axis=1)

In [ ]:
# tem_df.sort_values(by='sum', ascending=False)

In [ ]:
# топ 10
# recommended_items = tem_df.head(k)[itemIdColumnName].unique()

In [ ]:
# recommended_items

In [ ]:
# map_k_ItemsKNN_rand = map_k(recommended_items, random_user_interactions_test, k)

In [ ]:
# map_k_ItemsKNN_rand

# MTS RECTOOLS

In [ ]:
!pip install rectools

In [ ]:
!pip install git+https://github.com/daviddavo/lightfm

  Cloning https://github.com/daviddavo/lightfm to /tmp/pip-req-build-9q1acgoo
  Running command git clone --filter=blob:none --quiet https://github.com/daviddavo/lightfm /tmp/pip-req-build-9q1acgoo
  Resolved https://github.com/daviddavo/lightfm to commit f0eb500ead54ab65eb8e1b3890337a7223a35114
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp312-cp312-linux_x86_64.whl size=1099141 sha256=33311d005c99b988ee202be168156ff1811b42903dcd71bbc532b2405f4db41d
  Stored in directory: /tmp/pip-ephem-wheel-cache-j81h3usg/wheels/fd/89/93/70c1e5f378ee5043de89387ee3ef6852ff39e3b9eb44ecc1a3
Successfully built lightfm


In [ ]:
!pip install rectools[lightfm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.7/411.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rectools-lightfm: filename=rectools_lightfm-1.17.3-cp312-cp312-linux_x86_64.whl size=1074124 sha256=8100c3c44b347a233ee1d675d139d41111162820902e4b3cec015cbc768ceb4f
  Stored in directory: /root/.cache/pip/wheels/4f/c5/0d/0926e4783dab1901a3750d21a6f18e9589204ef55d747dad94
Successfully built rectools-lightfm


In [ ]:
import pandas as pd

from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import SASRecModel
from rectools.models import LightFMWrapperModel
from lightfm import LightFM


In [ ]:
model = LightFMWrapperModel(
        # внутри модели указываем параметр no_components
        # это размезность эмбеддингов, которые выучит модель
        epochs=100,
        model=LightFM(no_components = 30)
        )

In [ ]:
filtered_interactions_df = df_interactions_train.copy()
filtered_interactions_df.dropna(subset=[userColumnName, itemIdColumnName, watchdPrcntColumnName, dateColumnName], inplace=True)
filtered_interactions_df.reset_index()
filtered_interactions_df.drop(filtered_interactions_df.tail(1).index,inplace=True)

In [ ]:
filtered_interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2243502 entries, 2 to 5476243
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int64         
 1   item_id        int64         
 2   last_watch_dt  datetime64[ns]
 3   total_dur      int64         
 4   watched_pct    float64       
 5   reallyWatched  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3)
memory usage: 104.8 MB


In [ ]:


ratings = pd.DataFrame()
ratings[Columns.User] = filtered_interactions_df[userColumnName]
ratings[Columns.Item] = filtered_interactions_df[itemIdColumnName]
ratings[Columns.Weight] = filtered_interactions_df[watchdPrcntColumnName]
ratings[Columns.Datetime] = filtered_interactions_df[dateColumnName]


In [ ]:
dataset = Dataset.construct(ratings)


In [ ]:
users_uniq = filtered_user_tests_content_interaction[Columns.User].unique()

In [ ]:
print(dataset)

Dataset(user_id_map=IdMap(external_ids=array([ 656683,  864613,  648682, ...,  204332, 1007900,  882138])), item_id_map=IdMap(external_ids=array([ 7107,  7638,  1449, ..., 10064, 13516, 10542])), interactions=Interactions(df=         user_id  item_id  weight   datetime
0              0        0     0.0 2021-05-09
1              1        1   100.0 2021-07-05
2              2        2    75.0 2021-06-13
3              3        3   100.0 2021-07-07
4              4        4     1.0 2021-05-29
...          ...      ...     ...        ...
2243497    51618       43    28.0 2021-05-20
2243498   273545       19     8.0 2021-07-31
2243499   603782      216    79.0 2021-05-15
2243500   139252        0    75.0 2021-05-10
2243501    92153      938     1.0 2021-05-29

[2243502 rows x 4 columns]), user_features=None, item_features=None)


In [ ]:
model.fit(dataset)


In [ ]:

# Make recommendations
recos = model.recommend(
    users=users_uniq,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

In [ ]:
recos

,user_id,item_id,score,rank
0,106,9728,37.540924,1
1,106,4151,35.891125,2
2,106,10440,35.468941,3
3,106,4880,31.419081,4
4,106,8636,31.260462,5
...,...,...,...,...
18855,1094750,14431,6.769461,6
18856,1094750,4495,6.605996,7
18857,1094750,4685,6.171384,8
18858,1094750,11778,6.097526,9


In [ ]:
recos_df = recos.groupby(Columns.User)[Columns.Item].unique().reset_index()

In [ ]:
recos_df.columns = Columns.UserItem

In [ ]:
recos_df

,user_id,item_id
0,106,"[9728, 4151, 10440, 4880, 8636, 1844, 4495, 14..."
1,181,"[9728, 6809, 1844, 14431, 4495, 8636, 4685, 74..."
2,1180,"[10440, 9728, 6809, 4151, 1844, 14431, 4495, 4..."
3,2041,"[10440, 9728, 6809, 4151, 1844, 14431, 4495, 4..."
4,2094,"[10440, 9728, 6809, 4151, 1844, 14431, 4495, 4..."
...,...,...
1881,1094750,"[10440, 9728, 6809, 4151, 1844, 14431, 4495, 4..."
1882,1095299,"[10440, 9728, 6809, 4151, 1844, 14431, 4495, 4..."
1883,1095411,"[9728, 10440, 6809, 1844, 8636, 14431, 4880, 4..."
1884,1096807,"[9728, 4151, 10440, 6809, 4880, 8636, 1844, 14..."


In [ ]:
def getItemsName(unique_users,df, df_items, df_test_interac):
  sum_ap_k = 0
  u_len = len(unique_users)
  for u_id in unique_users:
    userRecs = getValueFromBy2Col(df, userColumnName, itemIdColumnName, u_id)
    test_first_user = getValueFromBy2Col(df_test_interac, userColumnName, itemIdColumnName, u_id)
    u_ap_k = ap_k(userRecs, test_first_user)
    sum_ap_k += u_ap_k
    # print(u_ap_k)
    if u_len <101:
      print("for user ", u_id,"actual: ")
      if test_first_user is not None:
        for i_id in test_first_user:
          itemName = getValueFromBy2Col(df_items, itemIdColumnName, "title", i_id)
          print(itemName)
        print("+++++++++++++++++++++++++++++++")
      else:
        print("None tests: ")

      if userRecs is not None:
        print("for user ", u_id,"recs: ")
        for i_id in userRecs:
          itemName = getValueFromBy2Col(df_items, itemIdColumnName, "title", i_id)
          print(itemName)
      else:
        print("None recs: ")
      print("____________________________________________________________________________________________________________________________________________________________________________________________")
  return sum_ap_k/u_len

In [ ]:
filtered_user_tests_content_interaction = filtered_user_tests_content_interaction.sort_values(Columns.Item, key=lambda s: [len(x) for x in s], ascending=False)


In [ ]:
filtered_user_tests_content_interaction

,user_id,item_id
14857,903689,"[14911, 5319, 6689, 5957, 8506, 7949, 8032, 10..."
11194,678785,"[13863, 5319, 954, 3138, 11006, 13468, 7084]"
1928,114794,"[383, 7214, 517, 11577, 3669, 12743]"
17009,1036171,"[517, 10119, 4495, 16420, 14222, 1811]"
903,54002,"[10119, 5589, 12743, 8986, 7789, 9170]"
...,...,...
6402,387876,"[7007, 8986]"
6400,387821,"[2954, 2100]"
6397,387590,"[144, 3587]"
6382,386715,"[3182, 12908]"


In [ ]:
users_to_watch = filtered_user_tests_content_interaction[Columns.User].unique()
# users_to_watch = filtered_user_tests_content_interaction.head(30)[Columns.User].unique()

In [ ]:
len(users_to_watch)

1886

In [ ]:
map_k_rectools = getItemsName(users_to_watch, recos_df, df_items, filtered_user_tests_content_interaction)

0
0
0
0.0
0
0
0.0
0
0
0
0
0.0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.0
0
0
0
0
0
0
0
0
0
0.25
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.0
0
0
0
0
0.0
0.0
0
0
0
0
0
0
0
0.0
0
0
0
0
0.0
0
0
0
0
0
0
0.0
0
0.0
0
0.0
0
0.0
0
0.0
0.0
0
0
0
0
0
0.0
0
0
0.0
0
0
0.0
0.0
0
0
0
0
0
0.0
0.0
0
0
0
0
0
0
0
0
0
0
0.0
0
0
0
0
0
0.0
0
0
0.0
0
0
0
0
0
0
0
0
0
0.0
0
0
0
0
0
0.0
0
0
0
0.0
0
0.0
0
0
0
0.0
0
0
0
0
0
0.0
0
0
0
0
0
0
0
0.0
0.125
0
0.0
0.0
0.0
0
0
0
0.25
0
0
0
0
0.0
0
0
0
0
0
0
0
0.0
0
0
0
0
0
0
0
0
0
0
0.0
0
0
0
0
0
0
0
0
0
0
0.1111111111111111
0.0
0
0
0
0.0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.3333333333333333
0
0
0
0.0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.0
0
0.0
0
0
0
0
0
0
0.0
0.25
0
0
0.0
0
0
0
0
0
0
0
0
0.0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.0
0
0
0
0.0
0
0
0.0
0
0
0
0
0
0
0
0
0.0
0
0
0.0
0
0
0
0
0.0
0
0
0.0
0
0
0
0.3333333333333333
0
0
0
0
0
0
0
0.0
0
0.0
0
0
0
0.0
0
0
0
0.0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.0
0.0
0
0
0.0
0.0
0.0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.0
0
0
0


In [ ]:
map_k_rectools

0.0033286202427241663